In [1]:
from google.colab import drive

drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install datasets transformers rouge_score nltk
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

### Imports

In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import re
import sys
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import random
import argparse
import textwrap
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [4]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [5]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [6]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text):
    # Convert all text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(tokens)

    # Remove extra whitespaces
    text = re.sub(' +', ' ', text)

    return text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Noise Functions

In [7]:
# Actions for char and word level noise functions
char_action = ['insert',
        'substitute',
        'delete',
        'swap',
]

word_action = ['substitute',
        'delete',
        'swap',
]


def get_action(type):
  if type=="char":
    return random.choice(char_action)
  elif type=="word":
    return random.choice(word_action)


def augment_tweet(tweet, p=0.7):
    """
    Augment a tweet with character-level and word-level noise.

    Args:
        tweet (str): The original tweet.
        p (float): The probability of applying the char level augmentation.

    Returns:
        str: The augmented tweet.
    """
    # Define a list of character-level augmentation techniques
    char_augmenters = [
        nac.OcrAug(),
        nac.KeyboardAug(aug_char_p=0.2, aug_word_p=0.2, include_special_char=False),
        nac.RandomCharAug(action=get_action("char"), aug_char_p=0.1, aug_word_p=0.1),
    ]

    # Define a list of word-level augmentation techniques
    word_augmenters = [
        naw.SpellingAug(),
        naw.SplitAug(),
        naw.SynonymAug(),
        naw.RandomWordAug(aug_p=0.2, action=get_action("word")),
    ]

    # Randomly apply a character-level or word-level augmentation with probability p
    if random.random() < p:
        aug = random.choice(char_augmenters)
        augmented_tweet = aug.augment(tweet)
    else:
        aug = random.choice(word_augmenters)
        augmented_tweet = aug.augment(tweet)
        
    return augmented_tweet[0]


In [8]:
def add_noise(df, augmentation_percentage, task):

  if task=="sentiment_analysis":
    # Sample 10% of the rows in the DataFrame
    augment_indices = df.sample(frac=augmentation_percentage).index

    # Apply the augment_tweet function to each tweet in the sampled rows
    for index in augment_indices:
        tweet = df.loc[index, 'text']
        augmented_tweet = augment_tweet(tweet)
        df.loc[index, 'text'] = augmented_tweet
    
    return df
  
  elif task=="question_answering":

    # TODO - noise functions for QA

    return df

## Data download and preprocess

In [9]:
# Download data
df = pd.read_csv('/content/gdrive/MyDrive/NLP Project/data/IMDB Dataset.csv')
df = df.rename(columns={'review':'text'})
df = df[['text', 'sentiment']]

df['text'] = df['text'].apply(clean_text)

# Find and delete any empty rows
empty_rows = df[df['text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0)]
df.drop(empty_rows.index, inplace=True)

# How much of the dataset to use
data_size = 0.2
df = df.sample(frac=data_size)

In [10]:
## Add word and char level noise
augmentation_percentage = 0.2
df = add_noise(df, augmentation_percentage, task="sentiment_analysis")

# Randomly shuffle all rows
# df = df.sample(frac=1).reset_index(drop=True)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [11]:
df.head()

,text,sentiment
33553,really liked summerslam due look arena curtain...,positive
9427,many television shows appeal quite many differ...,positive
199,film quickly gets major chase scene ever incre...,negative
12447,jane austen would definitely approve onebr br ...,positive
39489,expectations somewhat high went see movie thou...,negative


In [12]:
print("full data: ")
label_counts = df['sentiment'].value_counts()
print(label_counts)
print('\n\n ')

train_set, val_set = train_test_split(df, test_size=0.3, random_state=42)
val_set, test_set = train_test_split(val_set, test_size=0.33, random_state=42)

print("train: ")
label_counts = train_set['sentiment'].value_counts()
print(label_counts)
print('\n')

print("val: ")
label_counts = val_set['sentiment'].value_counts()
print(label_counts)
print('\n')

print("test: ")
label_counts = test_set['sentiment'].value_counts()
print(label_counts)
print('\n')

full data: 
positive    5039
negative    4961
Name: sentiment, dtype: int64


 
train: 
positive    3523
negative    3477
Name: sentiment, dtype: int64


val: 
positive    1019
negative     991
Name: sentiment, dtype: int64


test: 
positive    497
negative    493
Name: sentiment, dtype: int64




In [13]:
# Set the device as GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [14]:
ids_neg = tokenizer.encode('negative </s>')
ids_pos = tokenizer.encode('positive </s>')
ids_neg, ids_pos

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


([2841, 1], [1465, 1])

## Dataset and Dataloaders


In [15]:
# Define custom class for our dataset
class ImdbDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer, max_len=512):
    self.df = df
    self.tokenizer = tokenizer
    self.max_len = max_len
    
    self.inputs = []
    self.targets = []

    self._build()

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

  def _build(self):
    self._build_examples(self.df[self.df['sentiment'] == 'positive'], 'positive')
    self._build_examples(self.df[self.df['sentiment'] == 'negative'], 'negative')

    random.shuffle(self.inputs)
    random.shuffle(self.targets)
    
  def _build_examples(self, df, sentiment):
    
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

    for item in df['text']:
      
      text = item.strip()
      text = REPLACE_NO_SPACE.sub("", text) 
      text = REPLACE_WITH_SPACE.sub("", text)
      text = text + ' '

      target = sentiment + " "

      # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [text], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
      )
      # tokenize targets
      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target], max_length=2, pad_to_max_length=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)

In [16]:
# Build the dataset for each train, val and test data
train_dataset = ImdbDataset(train_set, tokenizer)
val_dataset = ImdbDataset(val_set, tokenizer)
test_dataset = ImdbDataset(test_set, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
len(train_dataset), len(val_dataset), len(test_dataset)

(7000, 2010, 990)

In [18]:
data = train_dataset[29]
print(tokenizer.decode(data['source_ids'], skip_special_tokens=True))
print(tokenizer.decode(data['target_ids'], skip_special_tokens=True))

early theres group suspiciously oldlooking teens theres maniac stalking around yes slashervillebr br movie called pranks called pranks havent faintest idea unless idea great prank repeatedly hit someones dinner baseball bat balance great prank fact quite rubbish prank truth told gobr br film concerns group teenagers tasked cleaning decommissioned dormitory become aware psychopath loose combat development split wander dark ends tears thembr br pranks badly made slasher movie dvd release viewed vipco one appears cut fair bit violence makes dvd even pointless lets face slasher movie shorn violence waste time slasherfilm video nasty completists
positive


In [19]:
args_dict = dict(
    model_name_or_path='t5-small',
    tokenizer_name_or_path='t5-small',
    max_seq_length=512,
    learning_rate=1e-5,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=32,
    eval_batch_size=32,
    num_train_epochs=5,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

params = argparse.Namespace(**args_dict)

In [20]:
# Define the optimizer and the learning rate scheduler
optimizer = AdamW(model.parameters(), lr=params.learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=params.warmup_steps, 
                                            num_training_steps=len(train_dataset)*10)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params.train_batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=params.eval_batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params.eval_batch_size, shuffle=True)

## Training



In [ ]:
# # Define the training loop
# best_val_loss = float("inf")
# best_epoch = 0
# epochs = 20

# for epoch in range(epochs):
#   # Train the model
#   model.train()
#   train_loop = tqdm(train_loader, desc=f'Training Epoch {epoch+1}', leave=True)

#   train_loss = 0.0

#   for batch in train_loop:
#     input_ids = batch["source_ids"].to(device)
#     attention_mask = batch["source_mask"].to(device)
#     lm_labels = batch["target_ids"].to(device)
#     decoder_attention_mask = batch["target_mask"].to(device)

#     outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=lm_labels, decoder_attention_mask=decoder_attention_mask)

#     loss = outputs['loss']

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     scheduler.step()

#     train_loss += loss.item()
#     train_loop.set_postfix(loss=loss.item())

#   train_loss /= len(train_loader)

#   # Evaluate the model on the validation set
#   model.eval()

#   val_loss = 0
#   with torch.no_grad():
#     for batch in val_loader:
#       input_ids = batch["source_ids"].to(device)
#       attention_mask = batch["source_mask"].to(device)
#       lm_labels = batch["target_ids"].to(device)
#       decoder_attention_mask = batch["target_mask"].to(device)

#       outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=lm_labels, decoder_attention_mask=decoder_attention_mask)

#       loss = outputs[0]
#       logits = outputs.logits.detach().cpu().numpy()

#       val_loss += loss.item()

#     val_loss /= len(val_loader)

#   if val_loss < best_val_loss:
#     best_epoch = epoch+1
#     best_val_loss = val_loss
#     # torch.save(model.state_dict(), "t5_sentiment_model.pt")
#     path = '/content/gdrive/MyDrive/NLP Project/models/T5_SA_noise20'

#     torch.save(model.state_dict(), path+'/model_parameters.pth')
  
#   print(f"Epoch {epoch + 1} - Val Loss: {val_loss:.4f}. \t Current best epoch is {best_epoch} with val loss - {best_val_loss:.4f}")

Training Epoch 1:   0%|          | 0/219 [00:00<?, ?it/s]

Epoch 1 - Val Loss: 0.7751. 	 Current best epoch is 1 with val loss - 0.7751


Training Epoch 2:   0%|          | 0/219 [00:00<?, ?it/s]

Epoch 2 - Val Loss: 0.4263. 	 Current best epoch is 2 with val loss - 0.4263


Training Epoch 3:   0%|          | 0/219 [00:00<?, ?it/s]

...

Epoch 18 - Val Loss: 0.3505. 	 Current best epoch is 18 with val loss - 0.3505


Training Epoch 19:   0%|          | 0/219 [00:00<?, ?it/s]

Epoch 19 - Val Loss: 0.3499. 	 Current best epoch is 19 with val loss - 0.3499


Training Epoch 20:   0%|          | 0/219 [00:00<?, ?it/s]

Epoch 20 - Val Loss: 0.3507. 	 Current best epoch is 19 with val loss - 0.3499


## CLEAN TESTING 

In [22]:
# Load model for evaluations
path = '/content/gdrive/MyDrive/NLP Project/models/T5_SA_noise20'
model.load_state_dict(torch.load(path+'/model_parameters.pth'))

<All keys matched successfully>

In [23]:
# Download data for Testing
df_test_clean = pd.read_csv('/content/gdrive/MyDrive/NLP Project/data/IMDB Dataset.csv')
df_test_clean = df_test_clean.rename(columns={'review':'text'})
df_test_clean = df_test_clean[['text', 'sentiment']]

df_test_clean['text'] = df_test_clean['text'].apply(clean_text)

# Find and delete any empty rows
empty_rows = df_test_clean[df_test_clean['text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0)]
df_test_clean.drop(empty_rows.index, inplace=True)

# How much of the dataset to use
data_size = 0.2
df_test_clean = df_test_clean.sample(frac=data_size)

train_set, val_set = train_test_split(df_test_clean, test_size=0.3, random_state=42)
val_set, test_set = train_test_split(val_set, test_size=0.33, random_state=42)

test_dataset_clean = ImdbDataset(test_set, tokenizer)
test_loader_clean = torch.utils.data.DataLoader(test_dataset_clean, batch_size=params.eval_batch_size, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
model.eval()
outputs = []
targets = []
for batch in test_loader_clean:
  outs = model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
  target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]
  outputs.extend(dec)
  targets.extend(target)

In [25]:
print(f"Accuracy on Cleaned Data: {metrics.accuracy_score(targets, outputs)}")

Accuracy on Cleaned Data: 0.48787878787878786


In [26]:
print(f"Classification Report on Cleaned Data: \n\n{metrics.classification_report(targets, outputs)}")

Classification Report on Cleaned Data: 

              precision    recall  f1-score   support

    negative       0.48      0.41      0.44       491
    positive       0.49      0.56      0.53       499

    accuracy                           0.49       990
   macro avg       0.49      0.49      0.48       990
weighted avg       0.49      0.49      0.48       990



## x% Noisy testing

In [27]:
# Download data for Testing
df_test_noisy = pd.read_csv('/content/gdrive/MyDrive/NLP Project/data/IMDB Dataset.csv')
df_test_noisy = df_test_noisy.rename(columns={'review':'text'})
df_test_noisy = df_test_noisy[['text', 'sentiment']]

df_test_noisy['text'] = df_test_noisy['text'].apply(clean_text)

# Find and delete any empty rows
empty_rows = df_test_noisy[df_test_noisy['text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0)]
df_test_noisy.drop(empty_rows.index, inplace=True)

# How much of the dataset to use
data_size = 0.2
df_test_noisy = df_test_noisy.sample(frac=data_size)

# Add NOISE
random.seed(42)
random_noise = random.uniform(0.05, 0.15)
print(f"Noise to be added: {random_noise}\n\n")

df_test_noisy = add_noise(df_test_noisy, augmentation_percentage=random_noise, task="sentiment_analysis")

train_set, val_set = train_test_split(df_test_noisy, test_size=0.3, random_state=42)
val_set, test_set = train_test_split(val_set, test_size=0.33, random_state=42)

test_dataset_noisy = ImdbDataset(test_set, tokenizer)
test_loader_noisy = torch.utils.data.DataLoader(test_dataset_noisy, batch_size=params.eval_batch_size, shuffle=True)

Noise to be added: 0.11394267984578837




/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [28]:
model.eval()
outputs = []
targets = []
for batch in test_loader_noisy:
  outs = model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
  target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]
  outputs.extend(dec)
  targets.extend(target)

In [29]:
print(f"Accuracy on {random_noise:.2f}% Noisy Data: {metrics.accuracy_score(targets, outputs)}")

Accuracy on 0.11% Noisy Data: 0.49797979797979797


In [30]:
print(f"Classification Report on {random_noise:.2f}% Noisy Data: \n\n{metrics.classification_report(targets, outputs)}")

Classification Report on 0.11% Noisy Data: 

              precision    recall  f1-score   support

    negative       0.47      0.39      0.42       470
    positive       0.52      0.60      0.56       520

    accuracy                           0.50       990
   macro avg       0.49      0.49      0.49       990
weighted avg       0.49      0.50      0.49       990



## Just making Predictions

In [ ]:
it = iter(val_loader)

In [ ]:
batch = next(it)
batch["source_ids"].shape

torch.Size([32, 512])

In [ ]:
outs = model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]

texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['source_ids']]
targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['target_ids']]

In [ ]:
for i in range(params.eval_batch_size):
    lines = textwrap.wrap("Review:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual sentiment: %s" % targets[i])
    print("Predicted sentiment: %s" % dec[i])
    print("=====================================================================\n")

Review: line good summary movie read watched minutes movie know exactly rest movie going behave
researcher named surgei pronounced sirgay searching hell underworld moscow however seems disappeared
friend oven follows team search could find boring plot bad actors looking old man underground acting
bad romance really feels pointless untrue often good reason way characters acting nothing scary
theres happening anything interesting whole entire movie walk asking gang leaders guys way walk
complain finds old man run finally finds way undergroundbr br thing provides giving movie lowest
ratings atmosphere want see movie may good choice dark mystical murky however rest movie really dull
sit time waiting movie begin something happening actually happening things well performed example
didnt really notice one characters disappeared swear wasnt sleeping something dont understand whats
dangerous children run around underground together scratching walls sticks suppose meant scary waybr
br watch movi